In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import pandas as pd
import os
import glob
import copy

from pprint import pprint
import itertools as it
from collections import Counter
import numpy as np

from aiida import load_profile
from aiida.orm.nodes.data.structure import StructureData

from aiida_quantumespresso.tools.pwinputparser import create_builder_from_file
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.calculations.pwimmigrant import PwimmigrantCalculation
# from aiida_quantumespresso.tools import pwinputparser
from aiida.engine import submit
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida.orm import load_node, load_code

from pymatgen.io.ase import AseAtomsAdaptor

# import pybat
# from pybat import Cathode, LiRichCathode, Dimer, DimerNEBAnalysis

from ase.visualize import view
from ase.io.vasp import read_vasp, write_vasp
from ase.build.tools import sort

from ase.io.cif import read_cif, write_cif
from ase.io.espresso import read_espresso_in, write_espresso_in

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)

In [2]:
load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

# Read in fully lithiated phospho-olivine and spinel structures from pw inputs

In [3]:
iurii_olivine_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii_unmod/LixMnPO4/Li1.00/DFT_plus_U/1_vcrelax/'
iurii_olivine_file = 'LiMnPO4.vcrelax.1.in'
iurri_spinel_dir = '/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn2O4/DFT+U/LiMn2O4/1_vcrelax'
iurii_spinel_file = 'LiMn2O4.vcrelax.1.in'

iurii_pseudo_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/Pseudopotentials/SSSP'

In [4]:

# ! Much simpler to just do it with ase-convert -> However, not sure if it would also complain about the special Mn and O namings
olivine_builder = create_builder_from_file(
    input_folder=iurii_olivine_dir,
    input_file_name=iurii_olivine_file,
    code='qe-6.4.1-pw',
    metadata={},
    pseudo_folder_path=iurii_pseudo_dir,
)

olivine_structure_data = olivine_builder['structure']

olivine_initial_ase = olivine_structure_data.get_ase()
olivine_initial_ase = sort(olivine_initial_ase)

olivine_initial_pmg = AseAtomsAdaptor.get_structure(olivine_initial_ase)


In [5]:
limnpo4 = read_espresso_in(os.path.join(iurii_olivine_dir, iurii_olivine_file))
view(limnpo4)

***
# Sampling of configurations using `icet`

In [19]:
import importlib
import pybat.core
_ = importlib.reload(pybat.core)

initial_cat = pybat.core.Cathode.from_structure(olivine_initial_pmg)

configuration_list = initial_cat.get_cation_configurations(
    substitution_sites=range(0,4),
    cation_list=["Li", "Vac"],
    sizes=[1, 2],
    # ! -> Up to twice the atom count, but also taking into account other symmetries
    concentration_restrictions={"Li": (0, 1)},
    max_configurations=None,
)

pmg_list = [cathode.as_ordered_structure() for cathode in configuration_list]
ase_list = [AseAtomsAdaptor.get_atoms(pmg_) for pmg_ in pmg_list]

# view(ase_list)

No magnetic moments found in structure, setting to zero.


In [20]:
# view(ase_list)

In [30]:
formula_list = [_.formula for _ in configuration_list]
multiplicity_dict = dict(zip(Counter(formula_list).keys(), Counter(formula_list).values()))

# Put all the data into pandas df
olivine_sampling_df = pd.DataFrame()

olivine_sampling_df['configuration'] = configuration_list
olivine_sampling_df['mn_number'] = [_.get_chemical_symbols().count('Mn') for _ in ase_list]
olivine_sampling_df['li_number'] = [_.get_chemical_symbols().count('Li') for _ in ase_list]
olivine_sampling_df['formula'] = [_.formula for _ in configuration_list]
olivine_sampling_df['cell'] = [np.array2string(_.cell.todict()['array']) for _ in ase_list]
olivine_sampling_df['space_group'] = [_.space_group for _ in configuration_list]
olivine_sampling_df['ase_structure'] = ase_list
olivine_sampling_df['lithiation'] = olivine_sampling_df['li_number'] / olivine_sampling_df['mn_number']

olivine_sampling_df = olivine_sampling_df.sort_values(by=['mn_number', 'lithiation'], ascending=[True, False])
olivine_sampling_df = olivine_sampling_df[['mn_number',   'li_number', 'lithiation', 'formula', 'space_group', 'cell', 'configuration', 'ase_structure']]

In [38]:
olivine_sampling_df.shape
olivine_sampling_df.head()
olivine_sampling_df.tail()

len(olivine_sampling_df['cell'].unique())

for column in ['mn_number', 'li_number', 'formula', 'cell', 'space_group']:

    print(column)
    pprint(olivine_sampling_df[column].value_counts().to_dict(), sort_dicts=False)

(472, 8)

,mn_number,li_number,lithiation,formula,space_group,cell,configuration,ase_structure
0,4,4,1.00,Li4 Mn4 P4 O16,Pnma,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[-2.92402818 -1.51445147 -4.58860127] Li, [-8...","(Atom('Li', [-2.9240281793305223, -1.514451474..."
1,4,3,0.75,Li3 Mn4 P4 O16,P-1,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[-8.1056484 -1.51447091 -2.22873108] Li, [-8...","(Atom('Li', [-8.105648398297449, -1.5144709080..."
2,4,3,0.75,Li3 Mn4 P4 O16,P-1,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[-2.92402818 -1.51445147 -4.58860127] Li, [-8...","(Atom('Li', [-2.9240281793305223, -1.514451474..."
3,4,2,0.50,Li2 Mn4 P4 O16,P2_1/c,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[-8.1056484 -4.54338086 -2.22873108] Li, [-2...","(Atom('Li', [-8.105648398297449, -4.5433808598..."
4,4,2,0.50,Li2 Mn4 P4 O16,P2_1/c,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[-8.1056484 -1.51447091 -2.22873108] Li, [-2...","(Atom('Li', [-8.105648398297449, -1.5144709080..."


,mn_number,li_number,lithiation,formula,space_group,cell,configuration,ase_structure
339,8,1,0.125,Li1 Mn8 P8 O32,P-1,[[-2.50872781e-06 0.00000000e+00 4.71973662e...,"[[2.25757633 7.57232268 2.49101209] Li, [7.439...","(Atom('Li', [2.257576327324794, 7.572322676014..."
404,8,1,0.125,Li1 Mn8 P8 O32,P-1,[[-2.50872781e-06 0.00000000e+00 4.71973662e...,"[[7.43919655 7.57230324 0.13114189] Li, [7.439...","(Atom('Li', [7.4391965462917184, 7.57230324242..."
405,8,1,0.125,Li1 Mn8 P8 O32,P-1,[[-2.50872781e-06 0.00000000e+00 4.71973662e...,"[[2.25757633 7.57232268 2.49101209] Li, [7.439...","(Atom('Li', [2.257576327324794, 7.572322676014..."
470,8,1,0.125,Li1 Mn8 P8 O32,P-1,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[17.80242629 1.51445147 -4.58858818] Li, [ 7...","(Atom('Li', [17.80242628936957, 1.514451474474..."
471,8,1,0.125,Li1 Mn8 P8 O32,P-1,[[ 2.50872781e-06 0.00000000e+00 -4.71973662e...,"[[12.62080607 1.51447091 -2.22871799] Li, [ 7...","(Atom('Li', [12.62080607040265, 1.514470908062..."


8

mn_number
{8: 462, 4: 10}
li_number
{4: 127, 3: 100, 5: 98, 2: 60, 6: 56, 1: 16, 7: 14, 0: 1}
formula
{'Li4 Mn8 P8 O32': 126,
 'Li5 Mn8 P8 O32': 98,
 'Li3 Mn8 P8 O32': 98,
 'Li6 Mn8 P8 O32': 56,
 'Li2 Mn8 P8 O32': 56,
 'Li7 Mn8 P8 O32': 14,
 'Li1 Mn8 P8 O32': 14,
 'Li2 Mn4 P4 O16': 4,
 'Li3 Mn4 P4 O16': 2,
 'Li1 Mn4 P4 O16': 2,
 'Li4 Mn4 P4 O16': 1,
 'Mn4 P4 O16': 1}
cell
{'[[ 0.00000000e+00 -6.05785177e+00  0.00000000e+00]\n [-5.01745562e-06  0.00000000e+00  9.43947324e+00]\n [-1.03632272e+01  0.00000000e+00 -6.54540506e-06]]': 66,
 '[[ 2.50872781e-06 -6.05785177e+00 -4.71973662e+00]\n [ 2.50872781e-06  6.05785177e+00 -4.71973662e+00]\n [ 1.03632272e+01  0.00000000e+00  6.54540506e-06]]': 66,
 '[[ 0.00000000e+00  6.05785177e+00  0.00000000e+00]\n [-5.01745562e-06  0.00000000e+00  9.43947324e+00]\n [ 1.03632297e+01  0.00000000e+00 -4.71973007e+00]]': 66,
 '[[-2.50872781e-06  6.05785177e+00  4.71973662e+00]\n [-2.50872781e-06 -6.05785177e+00  4.71973662e+00]\n [ 1.03632297e+01  0.000000

In [ ]:
olivine_sampling_df.loc[olivine_sampling_df['lithiation']]

In [32]:
view(ase_list)

<Popen: returncode: None args: ['/home/jgeiger/projects/bat_uv_ml/code/env/b...>

***
# AiiDA setup before LUMI-C access

In [11]:
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter

m = MPRester('aYd3U9UsKUuuvGvLcu9n8AEXeQaIXtlU')

si_bulk_pmg = m.get_structure_by_material_id('mp-149')


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:

si_bulk_structuredata = StructureData(pymatgen_structure=si_bulk_pmg)
si_bulk_ase = AseAtomsAdaptor.get_atoms(si_bulk_pmg)

si_test_builder = PwBaseWorkChain.get_builder_from_protocol(
    code='qe-6.4.1-pw',
    structure=si_bulk_structuredata,
)

# si_bulk_base_node = submit(si_test_builder)
# view(si_bulk_ase)
for k,v in si_test_builder.items():
    print(k, v)


metadata {}
pw {'metadata': {'options': {'stash': {}, 'resources': {'num_machines': 1}, 'max_wallclock_seconds': 43200, 'withmpi': True}}, 'monitors': {}, 'pseudos': {'Si': <UpfData: uuid: 23019765-e357-4fc8-b394-472defb13ace (pk: 20)>}, 'code': <InstalledCode: Remote code 'qe-6.4.1-pw' on theospc24 pk: 1, uuid: d54c69ca-1e3c-4492-99e7-cccb6262aa78>, 'structure': <StructureData: uuid: 93866dc8-3ceb-4572-ba67-3fa16f6dcfd1 (unstored)>, 'parameters': <Dict: uuid: 9ad053b9-6b82-4efe-aeef-65794f0c337a (unstored)>}
clean_workdir uuid: 28f54b01-24e1-426a-9c2b-4ea598afe087 (unstored) value: True
kpoints_distance uuid: 0334f75b-5a87-42d4-80be-368fe938553d (unstored) value: 0.15
kpoints_force_parity uuid: 236f4c10-50ef-49c0-9bb3-fb8fe44b40f3 (unstored) value: False


In [39]:
code = load_code('qe-6.4.1-pw')
builder = code.get_builder()

builder.code

# si_test_builder.structure?

<InstalledCode: Remote code 'qe-6.4.1-pw' on theospc24 pk: 1, uuid: d54c69ca-1e3c-4492-99e7-cccb6262aa78>

***
### Bash cell for verdi stuff

In [49]:
%%bash
# verdi process show 2171
# verdi process report 2171
verdi process list -a -p 1
# aiida load node
verdi computer show theospc24
verdi process show 2176

  PK  Created    Process label                 Process State     Process status
----  ---------  ----------------------------  ----------------  ----------------
2155  1h ago     PwBaseWorkChain               ⏹ Finished [300]
2156  1h ago     create_kpoints_from_distance  ⏹ Finished [0]
2160  1h ago     PwCalculation                 ⏹ Finished [305]
2171  57m ago    PwBaseWorkChain               ⏹ Finished [0]
2172  57m ago    create_kpoints_from_distance  ⏹ Finished [0]
2176  57m ago    PwCalculation                 ⏹ Finished [0]

Total results: 6

Report: last time an entry changed state: 53m ago (at 15:11:19 on 2023-02-13)
Report: Using 0% of the available daemon worker slots.
---------------------------  ------------------------------------
Label                        theospc24
PK                           1
UUID                         fa9496a0-ae25-4a61-972c-b55ba1dcbeb8
Description
Hostname                     localhost
Transport type               core.local
Scheduler type   

In [53]:
%%bash
%%capture
verdi calcjob inputls 2176    # shows the list of input files
verdi calcjob inputcat 2176   # shows the input file of the calculation
verdi calcjob outputls 2176   # shows the list of output files
# verdi calcjob outputcat 2176  # shows the output file of the calculation
verdi calcjob res 2176        # shows the parser results of the calculation

.aiida
_aiidasubmit.sh
aiida.in
&CONTROL
  calculation = 'scf'
  etot_conv_thr =   2.0000000000d-05
  forc_conv_thr =   1.0000000000d-04
  max_seconds =   4.1040000000d+04
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   1.0000000000d-02
  ecutrho =   2.4000000000d+02
  ecutwfc =   3.0000000000d+01
  ibrav = 0
  nat = 2
  nosym = .false.
  ntyp = 1
  occupations = 'smearing'
  smearing = 'cold'
/
&ELECTRONS
  conv_thr =   4.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Si     28.0855 Si.pbe-n-rrkjus_psl.1.0.0.UPF
ATOMIC_POSITIONS angstrom
Si           3.8891685000       2.7500585000       6.7362365000 
Si           0.5555955000       0.3928655000       0.9623195000 
K_POINTS automatic
14 14 14 0 0 0
CELL_PARAMETERS angstrom
      3.3335730000       0.0000000000       1.9246390000
      1.1111910000       3.1429240000       1.9246390000
      0

bash: line 1: fg: no job control


In [52]:
si_bulk_calc_node = load_node(2181)

si_bulk_calc_dict = si_bulk_calc_node.get_dict()
for k,v in si_bulk_calc_dict.items():
    print(k,v)
# verdi process show 2176


aiida.orm.nodes.data.dict.Dict

lsda False
energy -310.75143256422
volume 40.329526847414
degauss 0.136056917253
fft_grid [36, 36, 36]
energy_xc -167.34948603474
wall_time    3m34.04s 
rho_cutoff 3265.366014072
symmetries [{'t_rev': '0', 'symmetry_number': 0}, {'t_rev': '0', 'symmetry_number': 1}, {'t_rev': '0', 'symmetry_number': 32}, {'t_rev': '0', 'symmetry_number': 33}]
wfc_cutoff 408.170751759
exit_status 0
format_name QEXSD
lkpoint_dir False
occupations smearing
total_force 0.0
creator_name pwscf
energy_ewald -227.98593601969
energy_units eV
fermi_energy 6.1628
forces_units ev / angstrom
q_real_space False
stress_units GPascal
smearing_type mv
charge_density ./charge-density.dat
constraint_mag 0
energy_hartree 15.058102582342
format_version 19.03
scf_iterations 7
beta_real_space False
creator_version 6.4.1
energy_accuracy 1.1973008718264e-09
energy_smearing 0.00042544998025013
energy_xc_units eV
number_of_atoms 2
number_of_bands 8
smooth_fft_grid [25, 25, 30]
convergence_info {'scf_conv': {'scf_error': 8.779579

In [21]:
%%bash
verdi process show 2176
verdi process report 2176

Property     Value
-----------  ------------------------------------
type         PwCalculation
state        Finished [0]
pk           2176
uuid         3997a9bc-44ba-4bb4-9062-be9e16e5ff7c
label
description
ctime        2023-02-13 15:07:36.672014+01:00
mtime        2023-02-13 15:11:18.654284+01:00
computer     [1] theospc24

Inputs      PK    Type
----------  ----  -------------
pseudos
    Si      20    UpfData
code        1     InstalledCode
kpoints     2173  KpointsData
parameters  2174  Dict
settings    2175  Dict
structure   2165  StructureData

Outputs              PK  Type
-----------------  ----  --------------
output_band        2179  BandsData
output_parameters  2181  Dict
output_trajectory  2180  TrajectoryData
remote_folder      2177  RemoteData
retrieved          2178  FolderData

Caller          PK  Type
------------  ----  ---------------
iteration_01  2171  PwBaseWorkChain

Log messages
---------------------------------------------
There are 1 log messages for this cal

In [25]:
%%bash
verdi devel --help

Usage: verdi devel [OPTIONS] COMMAND [ARGS]...

  Commands for developers.

Options:
  -v, --verbosity [notset|debug|info|report|warning|error|critical]
                                  Set the verbosity of the output.
  -h, --help                      Show this message and exit.

Commands:
  check-load-time          Check for common indicators that slowdown...
  check-undesired-imports  Check that verdi does not import python...
  launch-add               Launch an ``ArithmeticAddCalculation``.
  rabbitmq                 Commands to interact with RabbitMQ.
  run-sql                  Run a raw SQL command on the profile database...
  validate-plugins         Validate all plugins by checking they can be...
